# 📈 Data Frequency and Timing Analysis from CSV
This notebook analyzes the timing consistency of logged data using timestamps from a `Millis` column in a CSV file.

We explore:
- 📊 The **instantaneous frequency** of data reception.
- ⏱️ The **time difference** (`Δt`) between consecutive samples.
- 📌 Basic **descriptive statistics** on frequency distribution.


> Units:
- Time difference is measured in **milliseconds (ms)**.
- Frequency is measured in **hertz (Hz)**.

---

📍 **Run this notebook on Google Colab**:  
👉 [Open in Colab](https://colab.research.google.com/drive/1u3gON0EEDnfnLpLB_TXoAjIaJn2-LgkI?usp=sharing)

---

💡 *Feel free to contribute suggestions or improvements! Your feedback is always welcome 😊*


In [46]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
import io

# Upload your CSV file
uploaded = files.upload()

# Automatically detect the uploaded file name
filename = next(iter(uploaded))

Saving sensor_data_2025-04-29_11-19-49.csv to sensor_data_2025-04-29_11-19-49 (3).csv


## ⏱️ Step 1: Calculate Time Differences and Frequencies

We compute:
- `Delta_t` = time difference between rows (in milliseconds)
- `Frequency_Hz` = 1000 / `Delta_t`

This shows how frequently the data was received at each sample point.


In [47]:
df = pd.read_csv(io.BytesIO(uploaded[filename]))

# Calculate Δt between entries
df['Delta_t'] = df['Millis'].diff()

# Calculate frequency: f = 1000 / Δt (in Hz)
df['Frequency_Hz'] = 1000 / df['Delta_t']


In [48]:
# Plot frequency
plt.figure(figsize=(12, 5))
plt.plot(df.index, df['Frequency_Hz'], marker='o', linestyle='-', color='green')
plt.title('Instantaneous Frequency of Data Reception')
plt.xlabel('Sample Index')
plt.ylabel('Frequency (Hz)')
plt.grid(True)
plt.show()

In [51]:
# Find all points where frequency is below or equal to 10 Hz
low_freq_df = df[df['Frequency_Hz'] <= 10]

# Print corresponding Millis values and their indexes
if not low_freq_df.empty:
    for idx in low_freq_df.index:
        t2 = df.loc[idx, 'Millis']
        t1 = df.loc[idx - 1, 'Millis'] if idx > 0 else None
        print(f"Low Frequency (~{df.loc[idx, 'Frequency_Hz']:.2f} Hz) at index {idx}:")
        print(f"    t1 (previous) = {t1}")
        print(f"    t2 (current)  = {t2}")
        print(f"    Δt = {df.loc[idx, 'Delta_t']} ms")
else:
    print("No frequencies below or equal to 10 Hz were found.")

Low Frequency (~0.88 Hz) at index 2840:
    t1 (previous) = 73355.0
    t2 (current)  = 74491.0
    Δt = 1136.0 ms


## 🔍 Step 3: Plot Time Difference (Δt) Between Data Points

This reveals irregularities or spikes in sampling delay.

In [49]:
# Drop the first row (diff creates NaN)
df = df.dropna().reset_index(drop=True)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Delta_t'], color='purple', marker='o', linestyle='-', markersize=2)

# Set plot labels and title
plt.title('Time Difference Between Data Points')
plt.xlabel('Sample Index')
plt.ylabel('Time Difference (ms)')
plt.ylim(0, df['Delta_t'].max() + 5)  # Y-axis from 0 to max delta + small margin
plt.grid(True)
plt.tight_layout()
plt.show()

## 📈 Step 4: Frequency Statistics

Get a quick overview of the frequency distribution.

In [50]:
# Optionally show basic stats
print("Time Difference Statistics:")
print(df['Frequency_Hz'].describe())

Time Difference Statistics:
count    7751.000000
mean       50.522912
std         3.696519
min         0.880282
25%        47.619048
50%        50.000000
75%        52.631579
max        58.823529
Name: Frequency_Hz, dtype: float64


## ⚠️ Observing Data Dropout or Loss

As seen in the plotted frequency graph, there are noticeable drops in data reception frequency — in some cases falling from stable values (e.g., ~50 Hz) down to **1 Hz or below**.

This behavior is typically referred to as:
- **Data dropout** or **data loss**: Indicates missing or delayed samples.

🔍 I am currently investigating these anomalies to determine their root cause and mitigate their effect on real-time data processing accuracy.
